#Implementation of Hand Recognition using Convolutional Neural Network for Gesture Language

Rian Rahman (L200164014)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Spesifikasi

In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
!lscpu |grep 'Model name'

Model name:                           Intel(R) Xeon(R) CPU @ 2.20GHz


In [5]:
!lscpu | grep 'Core(s) per socket:'

Core(s) per socket:                   1


In [6]:
!lscpu | grep 'Thread(s) per core'

Thread(s) per core:                   2


In [7]:
!free -h --si | awk  '/Mem:/{print $2}'

12G


In [8]:
!lscpu | grep "MHz"

#Split data into Train, Validation and Test

In [9]:
!pip install split-folders

In [10]:
## Tidak perlu dijalankan karena data suda displit
import splitfolders

input_folder = "/content/drive/MyDrive/Data" #Input Folder
output = "/content/drive/MyDrive/Skripsi" #Output Folder

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(0.8,0.2))

ValueError: The provided input folder "/content/drive/MyDrive/Data" does not exists.

#Data Preprocessing

In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras import optimizers, Sequential, models
import tensorflow as tf


In [11]:
#Parameter Augmentation data
IMG_SIZE = (64,64)
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)


NameError: name 'ImageDataGenerator' is not defined

In [ ]:
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Share/train',
                                                    target_size=IMG_SIZE,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical',
                                                    shuffle=True)

val_generator = val_datagen.flow_from_directory('/content/drive/MyDrive/Share/val',
                                                target_size=IMG_SIZE,
                                                batch_size=BATCH_SIZE,
                                                class_mode='categorical',
                                                shuffle=True)


#CNN Model

In [ ]:
#PARAMETER Model

EPOCH = 20
STEPS_EPOCH = len(train_generator)
VAL_STEPS = len(val_generator)
INPUT = (64,64,3)
print(STEPS_EPOCH, VAL_STEPS)


In [ ]:
model = Sequential([
    Conv2D(32, 3, activation='relu', input_shape=INPUT),
    MaxPool2D(),
    Conv2D(64, 3, activation='relu'),
    MaxPool2D(),
    Dropout(0),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(24, activation='softmax')
    ])
model.summary()

In [ ]:
from keras.utils import plot_model
model_save = plot_model(model,
                        to_file='model.png',
                        show_shapes=True,
                        rankdir='LR',
                        dpi=55)
model_save

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


Training by GPU

In [12]:
## Training using GPU
history = model.fit(
        train_generator,
        epochs = EPOCH,
        validation_data = val_generator,
        validation_steps = VAL_STEPS,
        shuffle=True)

NameError: name 'model' is not defined

Runtime 1:18:30

In [ ]:
# Loss (GPU)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.ylabel('value')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.show()


In [ ]:
# Accuracy (GPU)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('value')
plt.xlabel('epoch')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#save model
import os

MODEL_BASE_PATH = "/content/drive/MyDrive/model"
SAVE_MODEL_NAME = "model_2.h5"
save_model_path = os.path.join(MODEL_BASE_PATH,
                               SAVE_MODEL_NAME)

print('Saving Model At {}...'.format(save_model_path))
model.save(save_model_path)

In [ ]:
#Evaluasi model dengan data test

loss, acc = model.evaluate(train_generator,steps=len(val_generator),verbose=0)
print('Accuracy on training data: {:.4f} \nLoss on training data: {:.4f}'.format(acc,loss),'\n')

#eval = model.evaluate(x_train,y_train, varbose = 1)
#print('Accuracy on trainong data : {:.4} \nLoss on training data : {:.4}'.format(x_train,y_train,'\n')

Training by CPU

In [ ]:
## Training using CPU
history = model.fit(
        train_generator,
        epochs = EPOCH,
        validation_data = val_generator,
        validation_steps = VAL_STEPS,
        shuffle=True)

1:39:55 runtime CPU

# Prediction Model

In [ ]:
from keras.models import load_model
#from keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import numpy as np
import os
import pandas as pd

MODEL_PATH = '/content/drive/MyDrive/model/model_2.h5'
model = load_model(MODEL_PATH,compile=False)
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M',
          'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']

folder_path='/content/drive/MyDrive/Skripsi/val'
prediction = []
accuracy = []
target_name = []

In [ ]:
def preprocess(img,input_size):
  nimg = img.convert('RGB').resize(input_size, resample= 0)
  img_arr = (np.array(nimg))/255
  return img_arr

def reshapee(img_arr):
  return np.stack(img_arr, axis=0)

def predicting(x):
  y = model.predict(x)
  predict_y = labels[np.argmax(y)]
  prediction.append(predict_y)
  accuracy.append(np.max(y))

# read image
for fol in os.listdir(folder_path):
  for im in os.listdir(f"/content/drive/MyDrive/data/predict/{fol}") :
    img = Image.open(f"/content/drive/MyDrive/data/predict/{fol}/{im}")
    X = preprocess(img,(64,64))
    X = reshapee([X])
    predicting(X)
    target_name.append(fol)

In [ ]:
data_1 = {'Nama Data' : target_name,
        'Labels': prediction,
        'Accuracy': accuracy}

data_1 = pd.DataFrame(data_1)
data_1 = data_1.sort_values(by=['Nama Data'])
data_1 = data_1.reset_index(drop=True)
data_1['Accuracy'] = data_1['Accuracy']*100
data_1.info()

In [ ]:
data_1

In [ ]:
data_1['check'] = (data_1['Nama Data']== data_1['Labels'])

In [ ]:
data_1.to_csv('/content/drive/MyDrive/Skripsi/(Rev) Result Predition.csv')

In [ ]:
new_data = data_1.groupby('Nama Data').mean()
new_data

In [ ]:
new_sum = data_1.groupby('Nama Data').sum()
new_sum

In [ ]:
data = new_data
data['Count'] = new_sum['check']
data

In [ ]:
data.to_excel('/content/drive/MyDrive/Skripsi/(Rev) Result Predition.xlsx')